In [29]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


In [30]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [31]:
# Load the pre-trained VGG16 model without the top classification layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


80134624/80134624 [==============================] - 12s 0us/step


In [32]:
# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the transfer learning model
transfer_model = Model(inputs=base_model.input, outputs=output)


In [33]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


In [34]:
# Compile the model
transfer_model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

# Print the model summary to see the architecture
transfer_model.summary()



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0         
                                                                 
 block3_conv1 (Conv2D)       (None, 8, 8, 256)         295168    
          

In [35]:
# Start training the model on the CIFAR-10 dataset
epochs = 40
batch_size = 128
transfer_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

Epoch 1/40
391/391 [==============================] - 13s 26ms/step - loss: 1.4483 - accuracy: 0.4886 - val_loss: 1.3012 - val_accuracy: 0.5435
Epoch 2/40
391/391 [==============================] - 8s 22ms/step - loss: 1.2544 - accuracy: 0.5606 - val_loss: 1.2933 - val_accuracy: 0.5425
Epoch 3/40
391/391 [==============================] - 7s 19ms/step - loss: 1.1842 - accuracy: 0.5823 - val_loss: 1.2209 - val_accuracy: 0.5703
Epoch 4/40
391/391 [==============================] - 8s 19ms/step - loss: 1.1304 - accuracy: 0.6011 - val_loss: 1.1963 - val_accuracy: 0.5773
Epoch 5/40
391/391 [==============================] - 8s 19ms/step - loss: 1.0901 - accuracy: 0.6176 - val_loss: 1.1913 - val_accuracy: 0.5842
Epoch 6/40
391/391 [==============================] - 8s 19ms/step - loss: 1.0533 - accuracy: 0.6301 - val_loss: 1.1968 - val_accuracy: 0.5798
Epoch 7/40
391/391 [==============================] - 8s 20ms/step - loss: 1.0210 - accuracy: 0.6423 - val_loss: 1.1769 - val_accuracy: 0.587

In [36]:
# Evaluate the model on the test set
score = transfer_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 1.5573316812515259
Test accuracy: 0.6047999858856201
